In [1]:
!pip install faker


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import pyspark
from faker import Faker
from pyspark.sql import SparkSession, DataFrame, functions as F, types as t
from pyspark.sql.window import Window
import random
from datetime import datetime, timedelta, date

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("pyspark-test01") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()
print(spark.catalog.listDatabases())

fake = Faker()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 13:57:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/02 13:57:10 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/02 13:57:10 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/02 13:57:13 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/02 13:57:13 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore codespace@172.17.0.3


[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/workspaces/devcontainer-universal/spark-warehouse')]


24/04/02 13:57:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [39]:
@F.udf(t.IntegerType())
def f_int(min:int, max:int) -> int:
    return fake.random_int(min=min, max=max)

@F.udf(t.IntegerType())
def f_intUnique(min:int, max:int) -> int:
    return fake.unique.random_int(min=min, max=max)

@F.udf(t.DecimalType())
def f_decimal(min:int, max:int) -> float:
    return fake.pydecimal(left_digits=12, right_digits=2, positive = True, min_value=min, max_value=max)

@F.udf(t.StringType())
def f_choise(list:list, weights:list) -> str:
    return random.choices(list,weights=weights)[0]

@F.udf(t.StringType())
def f_choise_active() -> str:
    return random.choices(['Active', 'Inactive'],weights=[0.8,0.2])[0]

@F.udf(t.BooleanType())
def f_bool() -> bool:
    return fake.boolean()

@F.udf(t.StringType())
def f_GUID() -> str:
    return fake.uuid4()

@F.udf(t.StringType())
def f_hash() -> str:
    return fake.md5(raw_output=False)

@F.udf(t.StringType())
def f_date(start:str='-3y', end:str='now') -> str:
    return str(fake.date_between(start_date=start, end_date=end))

@F.udf(t.StringType())
def f_timestamp(start:str='-3y', end:str='now') -> str:
    return str(fake.date_time_between(start_date=start, end_date=end))


In [93]:
Faker.seed(42)
random.seed(42)

consumers = spark.read.table('master_registry').select('Source_Consumer_Key', 'Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key')
# consumers.count()
'''
Transaction_Header_Key	BIGINT
Division_Code_Key	BIGINT
Region_Code_Key	BIGINT
Affiliate_Code_Key	BIGINT
Market_Code_Key	BIGINT
Source_Consumer_Key	BIGINT
Cfdm_Source_System_Code_Key	BIGINT
Customer_Door_Identifier_Key	BIGINT
Local_Currency_Code_Key	BIGINT
Date_Key	INT
Transaction_Header_Cfdm_Channel_Code_Key	BIGINT
Transaction_Header_Source_Transaction_Identifier	STRING
Transaction_Header_Source_Store_Identifier	STRING
Transaction_Header_Transaction_Timestamp	TIMESTAMP
Transaction_Header_Pos_Register_Identifier	STRING
Transaction_Header_Transaction_Type_Name	STRING
Transaction_Header_Consumer_Full_Name	STRING
Transaction_Header_Consumer_Email_Address	STRING
Transaction_Header_Online_Billing_Line_1_Address	STRING
Transaction_Header_Online_Billing_Line_2_Address	STRING
Transaction_Header_Online_Billing_Line_3_Address	STRING
Transaction_Header_Online_Billing_City_Name	STRING
Transaction_Header_Online_Billing_Country_Name	STRING
Transaction_Header_Online_Billing_Postal_Code	STRING
Transaction_Header_Tourist_Order_Indicator	TINYINT
Transaction_Header_Employee_Order_Indicator	TINYINT
Transaction_Header_Guest_Order_Indicator	TINYINT
Transaction_Header_Consumer_Capture_Indicator	TINYINT
Transaction_Header_Consumer_Mac_Pro_Indicator	TINYINT
Transaction_Header_Loyalty_Transaction_Indicator	TINYINT
Transaction_Header_Total_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Saleable_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Shipping_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Discount_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Returns_Amount	DECIMAL(142)
Calc_Transaction_Header_Total_Service_Amount	DECIMAL(142)
Transaction_Header_Total_Quantity	INT
Calc_Transaction_Header_Total_Saleable_Quantity	INT
Calc_Transaction_Header_Total_Shipping_Quantity	INT
Calc_Transaction_Header_Total_Discount_Quantity	INT
Calc_Transaction_Header_Total_Return_Quantity	INT
Calc_Transaction_Header_Total_Service_Quantity	INT
Calc_Transaction_Header_Total_Sample_Quantity	INT
Calc_Transaction_Header_Total_Gift_Quantity	INT
Calc_Transaction_Header_Distinct_Sku_Count	INT
Transaction_Header_Record_Created_Timestamp	TIMESTAMP
Transaction_Header_Record_Modified_Timestamp	TIMESTAMP
Transaction_Header_Source_Consumer_Identifier	STRING
'''
tmp_df = (consumers
          .withColumn('Transaction_Header_Key', f_intUnique(F.lit(1),F.lit(1000000)))  #	BIGINT
          .withColumn('Cfdm_Source_System_Code_Key', F.lit(1)) #	BIGINT
          .withColumn('Customer_Door_Identifier_Key', F.lit(1))  #	BIGINT
          .withColumn('Local_Currency_Code_Key', F.lit(1)) #	BIGINT
          .withColumn('Date_Key', F.date_format(f_date(), 'yyyyddMM').cast('integer'))  #	INT
          .withColumn('Transaction_Header_Cfdm_Channel_Code_Key', F.lit(1))  #	BIGINT
          .withColumn('Transaction_Header_Source_Transaction_Identifier', f_GUID())  #	STRING
          .withColumn('Transaction_Header_Source_Store_Identifier', f_GUID())  #	STRING
          .withColumn('Transaction_Header_Transaction_Timestamp', F.to_timestamp(f_timestamp()))  #	TIMESTAMP
          .withColumn('Transaction_Header_Pos_Register_Identifier', f_GUID())  #	STRING
          .withColumn('Transaction_Header_Transaction_Type_Name', F.lit('String'))  #	STRING
          .withColumn('Transaction_Header_Consumer_Full_Name', F.lit(None)) #	STRING
          .withColumn('Transaction_Header_Consumer_Email_Address', F.lit(None)) #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_1_Address', F.lit(None))  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_2_Address', F.lit(None))  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Line_3_Address', F.lit(None))  #	STRING
          .withColumn('Transaction_Header_Online_Billing_City_Name', F.lit(None)) #	STRING
          .withColumn('Transaction_Header_Online_Billing_Country_Name', F.lit(None))  #	STRING
          .withColumn('Transaction_Header_Online_Billing_Postal_Code', F.lit(None)) #	STRING
          .withColumn('Transaction_Header_Tourist_Order_Indicator', f_bool().cast('integer'))  #	TINYINT
          .withColumn('Transaction_Header_Employee_Order_Indicator', f_bool().cast('integer')) #	TINYINT
          .withColumn('Transaction_Header_Guest_Order_Indicator', f_bool().cast('integer'))  #	TINYINT
          .withColumn('Transaction_Header_Consumer_Capture_Indicator', f_bool().cast('integer')) #	TINYINT
          .withColumn('Transaction_Header_Consumer_Mac_Pro_Indicator', f_bool().cast('integer')) #	TINYINT
          .withColumn('Transaction_Header_Loyalty_Transaction_Indicator', f_bool().cast('integer'))  #	TINYINT
          .withColumn('Transaction_Header_Total_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Calc_Transaction_Header_Total_Saleable_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Calc_Transaction_Header_Total_Shipping_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Calc_Transaction_Header_Total_Discount_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Calc_Transaction_Header_Total_Returns_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Calc_Transaction_Header_Total_Service_Amount', f_decimal(F.lit(1),F.lit(100)))  # DECIMAL(14,2)
          .withColumn('Transaction_Header_Total_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Saleable_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Shipping_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Discount_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Return_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Service_Quantity', f_int(F.lit(1),F.lit(5)))  #	INT
          .withColumn('Calc_Transaction_Header_Total_Sample_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Total_Gift_Quantity', f_int(F.lit(1),F.lit(5))) #	INT
          .withColumn('Calc_Transaction_Header_Distinct_Sku_Count', f_int(F.lit(1),F.lit(5)))  #	INT
          .withColumn('Transaction_Header_Record_Created_Timestamp', F.to_timestamp(f_timestamp())) #	TIMESTAMP
          .withColumn('Transaction_Header_Record_Modified_Timestamp', F.to_timestamp(f_timestamp()))  #	TIMESTAMP
          .withColumn('Transaction_Header_Source_Consumer_Identifier', f_GUID()) #	STRING
)
tmp_df.show()


+-------------------+-----------------+---------------+------------------+---------------+----------------------+---------------------------+----------------------------+-----------------------+--------+----------------------------------------+------------------------------------------------+------------------------------------------+----------------------------------------+------------------------------------------+----------------------------------------+-------------------------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+-------------------------------------------+----------------------------------------------+---------------------------------------------+------------------------------------------+-------------------------------------------+----------------------------------------+---------------------------------------------+-

In [60]:
spark.sql('drop table if exists master_registry;')
          
spark.sql('''
CREATE TABLE master_registry (
  Consumer_Master_Registry_Key BIGINT NOT NULL,
  Region_Code_Key BIGINT,
  Affiliate_Code_Key BIGINT,
  Market_Code_Key BIGINT,
  Division_Code_Key BIGINT,
  Source_Consumer_Key BIGINT NOT NULL,
  Consumer_Master_Registry_Record_Valid_From_Timestamp TIMESTAMP NOT NULL,
  Consumer_Master_Level_Code STRING NOT NULL,
  Consumer_Master_Registry_Record_Valid_To_Timestamp TIMESTAMP NOT NULL,
  Consumer_Master_Registry_Global_Unique_Identifier STRING NOT NULL,
  Consumer_Master_Registry_Current_Record_Indicator TINYINT NOT NULL,
  Consumer_Master_Registry_Record_Created_Timestamp TIMESTAMP NOT NULL,
  Consumer_Master_Registry_Record_Modified_Timestamp TIMESTAMP NOT NULL);
''')

24/04/02 16:18:14 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/04/02 16:18:14 WARN HiveMetaStore: Location: file:/workspaces/devcontainer-universal/spark-warehouse/master_registry specified for non-external table:master_registry


DataFrame[]

In [124]:
spark.sql('select * from consumer_dimension').printSchema()

root
 |-- Consumer_Key: integer (nullable = true)
 |-- Consumer_Created_Source_System_Key: integer (nullable = true)
 |-- Consumer_Last_Update_Source_System_Key: integer (nullable = true)
 |-- Consumer_Web_Push_Opt_In_Opt_Out_Source_System_Key: integer (nullable = true)
 |-- Consumer_Privacy_Policy_Consent_Source_Key: integer (nullable = true)
 |-- Consumer_Class_Code_Key: integer (nullable = true)
 |-- CFDM_Channel_Code_Key: integer (nullable = true)
 |-- Consumer_Gender_Code_Key: integer (nullable = true)
 |-- Region_Code_Key: integer (nullable = true)
 |-- Affiliate_Code_Key: integer (nullable = true)
 |-- Market_Code_Key: integer (nullable = true)
 |-- Division_Code_Key: integer (nullable = true)
 |-- Consumer_Global_Unique_Identifier: string (nullable = true)
 |-- Consumer_Level_Code: string (nullable = true)
 |-- Consumer_Valid_Record_Indicator: integer (nullable = true)
 |-- Consumer_Version_Number: integer (nullable = true)
 |-- Consumer_Offline_Identifier: string (nullable = t

In [116]:
cons = spark.read.csv('src/master_reg.csv', header=True, inferSchema=True)
cons.write.insertInto('master_registry', overwrite=True)
spark.sql('select * from master_registry').count()

1000

In [92]:
# spark.catalog.listTables()
tmp_df.select(
    'Transaction_Header_Key',
    'Division_Code_Key',
    'Region_Code_Key',
    'Affiliate_Code_Key',
    'Market_Code_Key',
    'Source_Consumer_Key',
    'Cfdm_Source_System_Code_Key',
    'Customer_Door_Identifier_Key',
    'Local_Currency_Code_Key',
    'Date_Key',
    'Transaction_Header_Cfdm_Channel_Code_Key',
    'Transaction_Header_Source_Transaction_Identifier',
    'Transaction_Header_Source_Store_Identifier',
    'Transaction_Header_Transaction_Timestamp',
    'Transaction_Header_Pos_Register_Identifier',
    'Transaction_Header_Transaction_Type_Name',
    'Transaction_Header_Consumer_Full_Name',
    'Transaction_Header_Consumer_Email_Address',
    'Transaction_Header_Online_Billing_Line_1_Address',
    'Transaction_Header_Online_Billing_Line_2_Address',
    'Transaction_Header_Online_Billing_Line_3_Address',
    'Transaction_Header_Online_Billing_City_Name',
    'Transaction_Header_Online_Billing_Country_Name',
    'Transaction_Header_Online_Billing_Postal_Code',
    'Transaction_Header_Tourist_Order_Indicator',
    'Transaction_Header_Employee_Order_Indicator',
    'Transaction_Header_Guest_Order_Indicator',
    'Transaction_Header_Consumer_Capture_Indicator',
    'Transaction_Header_Consumer_Mac_Pro_Indicator',
    'Transaction_Header_Loyalty_Transaction_Indicator',
    'Transaction_Header_Total_Amount',
    'Calc_Transaction_Header_Total_Saleable_Amount',
    'Calc_Transaction_Header_Total_Shipping_Amount',
    'Calc_Transaction_Header_Total_Discount_Amount',
    'Calc_Transaction_Header_Total_Returns_Amount',
    'Calc_Transaction_Header_Total_Service_Amount',
    'Transaction_Header_Total_Quantity',
    'Calc_Transaction_Header_Total_Saleable_Quantity',
    'Calc_Transaction_Header_Total_Shipping_Quantity',
    'Calc_Transaction_Header_Total_Discount_Quantity',
    'Calc_Transaction_Header_Total_Return_Quantity',
    'Calc_Transaction_Header_Total_Service_Quantity',
    'Calc_Transaction_Header_Total_Sample_Quantity',
    'Calc_Transaction_Header_Total_Gift_Quantity',
    'Calc_Transaction_Header_Distinct_Sku_Count',
    'Transaction_Header_Record_Created_Timestamp',
    'Transaction_Header_Record_Modified_Timestamp',
    'Transaction_Header_Source_Consumer_Identifier'
).write.insertInto('transaction_header', overwrite=False)
spark.sql('select * from transaction_header').count()

4000

In [128]:
groupBy_columns = ['Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key']

# Total Buyers, will join it to Consumer Dimention on Consumer Master Reg GUID
trans_head = spark.read.table('transaction_header').select('Source_Consumer_Key')
master_reg = spark.read.table('master_registry').select('Source_Consumer_Key', 'Consumer_Master_Registry_Global_Unique_Identifier')
total_buyers = (
    master_reg
    .join(trans_head, master_reg.Source_Consumer_Key == trans_head.Source_Consumer_Key, 'left')
    .drop(trans_head.Source_Consumer_Key)
)
total_buyers.show()

+-------------------+-------------------------------------------------+
|Source_Consumer_Key|Consumer_Master_Registry_Global_Unique_Identifier|
+-------------------+-------------------------------------------------+
|          749621471|                             050b59b9-2835-482...|
|          749621471|                             050b59b9-2835-482...|
|          749621471|                             050b59b9-2835-482...|
|          749621471|                             050b59b9-2835-482...|
|          453778757|                             4c573d1e-3f5b-4ac...|
|          453778757|                             4c573d1e-3f5b-4ac...|
|          453778757|                             4c573d1e-3f5b-4ac...|
|          453778757|                             4c573d1e-3f5b-4ac...|
|          365341214|                             84605a17-1b6d-4d4...|
|          365341214|                             84605a17-1b6d-4d4...|
|          365341214|                             84605a17-1b6d-

In [120]:
# Lifecycle
lifecycle = (
    spark.read.table('consumer_lifecycle')
    .select('Consumer_Key','Consumer_Lifecycle_Status_Code_Key')
    .where(F.col('Consumer_Lifecycle_Status_Current_Record_Indicator') == 1)
)
lifecycle.show()

+------------+----------------------------------+
|Consumer_Key|Consumer_Lifecycle_Status_Code_Key|
+------------+----------------------------------+
|        1701|                                 1|
|       23735|                                 4|
|       35873|                                 2|
|       43076|                                 2|
|       43219|                                 6|
|       48349|                                 2|
|       50710|                                 2|
|       52774|                                 1|
|       71603|                                 2|
|       79415|                                 1|
|       79470|                                 2|
|       81071|                                 1|
|      101757|                                 6|
|      103119|                                 3|
|      109751|                                 6|
|      120461|                                 2|
|      127543|                                 2|


In [153]:
# Loyalty
loyalty = spark.read.table('consumer_loyalty').select('Consumer_Key','Consumer_Loyalty_Membership_Identifier').filter(F.col('Consumer_Loyalty_Active_Indicator') == 1)
loyalty.orderBy('Consumer_Key').show()

+------------+--------------------------------------+
|Consumer_Key|Consumer_Loyalty_Membership_Identifier|
+------------+--------------------------------------+
|         425|                  20509c62-08e8-45d...|
|        3402|                  74962764-12a4-4ef...|
|        6182|                  811414f8-d9df-4d0...|
|        7540|                  a9f948b2-4e63-44b...|
|        9767|                  f1738856-e25d-46e...|
|       20324|                  fe54ee4b-80ff-417...|
|       20480|                  2fb21e66-7ed1-4aa...|
|       25611|                  39b86bb2-b596-4a7...|
|       26225|                  bd9c66b3-ad3c-4d6...|
|       28418|                  0f6ce9bf-fe7e-436...|
|       28941|                  70cafdd8-fc04-4f0...|
|       31244|                  e2acf72f-9e57-4f7...|
|       32938|                  bd1531c8-3764-4bd...|
|       33326|                  4b0dbb41-8d52-48f...|
|       33659|                  3e896c64-e117-4ac...|
|       40115|              

In [132]:
# Contactables
phone = spark.read.table('consumer_phone').select('Consumer_Key','Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Phone_Current_Record_Indicator') == 1)
email = spark.read.table('consumer_email').select('Consumer_Key','Consumer_Email_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Email_Current_Record_Indicator') == 1)
# address = spark.read.table('consumer_address').select('Consumer_Key','Consumer_Address_Postal_Opt_In_Opt_Out_Indicator','Consumer_Address_Current_Record_Indicator').where(F.col('Consumer_Address_Current_Record_Indicator')==1)
consumers = spark.read.table('consumer_dimension').select('Consumer_Key', 'Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key')
contactable = (consumers
               .join(phone, consumers.Consumer_Key == phone.Consumer_Key, 'left').drop(phone.Consumer_Key)
               .join(email, consumers.Consumer_Key == email.Consumer_Key, 'left').drop(email.Consumer_Key)
            #    .join(address, consumers.Consumer_Key == address.Consumer_Key, 'left').drop(address.Consumer_Key)
)
contactable.show()


+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+
|Consumer_Key|Division_Code_Key|Region_Code_Key|Affiliate_Code_Key|Market_Code_Key|Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator|Consumer_Email_Opt_In_Opt_Out_Indicator|
+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+
|        1701|                1|             10|            886963|              1|                                          0|                                   NULL|
|       23735|                3|             10|            184991|              1|                                          1|                                   NULL|
|       35873|               10|             10|            588046|              1|                                       NULL|                                 

In [125]:
spark.read.table('consumer_dimension').printSchema()

root
 |-- Consumer_Key: integer (nullable = true)
 |-- Consumer_Created_Source_System_Key: integer (nullable = true)
 |-- Consumer_Last_Update_Source_System_Key: integer (nullable = true)
 |-- Consumer_Web_Push_Opt_In_Opt_Out_Source_System_Key: integer (nullable = true)
 |-- Consumer_Privacy_Policy_Consent_Source_Key: integer (nullable = true)
 |-- Consumer_Class_Code_Key: integer (nullable = true)
 |-- CFDM_Channel_Code_Key: integer (nullable = true)
 |-- Consumer_Gender_Code_Key: integer (nullable = true)
 |-- Region_Code_Key: integer (nullable = true)
 |-- Affiliate_Code_Key: integer (nullable = true)
 |-- Market_Code_Key: integer (nullable = true)
 |-- Division_Code_Key: integer (nullable = true)
 |-- Consumer_Global_Unique_Identifier: string (nullable = true)
 |-- Consumer_Level_Code: string (nullable = true)
 |-- Consumer_Valid_Record_Indicator: integer (nullable = true)
 |-- Consumer_Version_Number: integer (nullable = true)
 |-- Consumer_Offline_Identifier: string (nullable = t

In [157]:
consumer_dim = spark.read.table('consumer_dimension').select(
    'Consumer_Key','Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key','Consumer_Global_Unique_Identifier','Consumer_Anonymization_Indicator','CFDM_Channel_Code_Key'
)
df_joined = (
    consumer_dim
    .join(contactable, consumer_dim.Consumer_Key == contactable.Consumer_Key, 'left')
    .join(loyalty, consumer_dim.Consumer_Key == loyalty.Consumer_Key, 'left')
    # .join(total_buyers, consumer_dim.Consumer_Global_Unique_Identifier == total_buyers.Consumer_Master_Registry_Global_Unique_Identifier, 'left')
             
)
df_joined = (df_joined
            .groupBy(
                 consumer_dim.Division_Code_Key, 
                 consumer_dim.Region_Code_Key,
                #  consumer_dim.Affiliate_Code_Key,
                 consumer_dim.Market_Code_Key
            )
            .agg(
                F.countDistinct(consumer_dim.Consumer_Key).alias('Consumer Aggregate Total Consumer Count'),
                F.count_if((contactable.Consumer_Email_Opt_In_Opt_Out_Indicator == 1) |
                           (contactable.Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator == 1)
                           ).alias('Consumer Aggregate Contactable Consumer Count'),
                F.count_if(contactable.Consumer_Email_Opt_In_Opt_Out_Indicator == 1).alias('Consumer Aggregate Email Opt In Consumer Count'),
                F.count_if(contactable.Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator == 1).alias('Consumer Aggregate Sms Opt In Consumer Count'),
                F.count(loyalty.Consumer_Loyalty_Membership_Identifier).alias('Consumer Aggregate Loyalty Consumer Count'),
                F.count_if(consumer_dim.CFDM_Channel_Code_Key == 0).alias('Consumer Aggregate Online Exclusive Consumer Count'),
                F.count_if(consumer_dim.CFDM_Channel_Code_Key == 1).alias('Consumer Aggregate Offline Exclusive Consumer Count'),
                F.count_if(consumer_dim.CFDM_Channel_Code_Key == 2).alias('Consumer Aggregate Omnichannel Consumer Count'),
                F.count_if(consumer_dim.Consumer_Anonymization_Indicator == 1).alias('Consumer Aggregate Anonymized Consumer Count'),
            )
)
df_joined.show()

+-----------------+---------------+---------------+---------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------+-----------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+--------------------------------------------+
|Division_Code_Key|Region_Code_Key|Market_Code_Key|Consumer Aggregate Total Consumer Count|Consumer Aggregate Contactable Consumer Count|Consumer Aggregate Email Opt In Consumer Count|Consumer Aggregate Sms Opt In Consumer Count|Consumer Aggregate Loyalty Consumer Count|Consumer Aggregate Online Exclusive Consumer Count|Consumer Aggregate Offline Exclusive Consumer Count|Consumer Aggregate Omnichannel Consumer Count|Consumer Aggregate Anonymized Consumer Count|
+-----------------+---------------+---------------+-----------------